In [31]:
# Build a QVM (Quality, Value, Momentum) ranking system and see the returns by decile

from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.factors import Returns

In [32]:
def min_liq_univ():
    # replicate P123 universe "Minimum Liquidity NO OTC - Copy"
    # for 2005-01-03, this screen returns 3782 stocks.  P123 returns 4260 stocks.
    # for 2010-01-04, this screen returns 3543 stocks.  P123 returns 3686 stocks.
    # for 2016-01-01, this screen returns 3999 stocks.  P123 returns 3901 stocks.
    # for 2018-06-25, this screen returns 4428 stocks.  P123 returns 3894 stocks.
    
    # markt cap screen
    mktcap = Fundamentals.market_cap.latest
    mktcap_screen = mktcap > 50000000
    
    # price screen
    price_close = USEquityPricing.close.latest
    price_screen = price_close > 1
    
    # volume screen
    dollar_volume = AverageDollarVolume(window_length=20)
    volume_screen = dollar_volume > 200000
    
    # not OTC
    not_otc = ~mstar.share_class_reference.exchange_id.latest.startswith('OTC')
    
    return (mktcap_screen & price_screen & volume_screen & not_otc)
    

In [78]:
def make_pipeline():
    # get factors
    mktcap = Fundamentals.market_cap.latest
    price_close = USEquityPricing.close.latest
    
    # get factors, higher is better
    roe = Fundamentals.roe.latest
    fcf_yield = Fundamentals.fcf_yield.latest
    ret_20 = Returns(window_length = 20)
    
    # compare built-in valuation ratio with manually calculated
    price_close = USEquityPricing.close.latest
    fcf_per_share = Fundamentals.fcf_per_share.latest
    fcf_yield_manual = fcf_per_share / price_close
    
    # get ranks, masked by the universe, higher is better
    quality = roe.rank(method = 'average', mask = min_liq_univ())
    value = fcf_yield.rank(method = 'average', mask = min_liq_univ())
    momentum = ret_20.rank(method = 'average', mask = min_liq_univ())
    
    # combine ranks
    qvm = (quality + value + momentum).rank(method = 'average', mask = min_liq_univ())

    
    return Pipeline(
        columns={
            'mktcap': mktcap,
            'fcf_yield': fcf_yield,
            'fcf_yield_manual': fcf_yield_manual,
            'roe': roe,
            'ret_20': ret_20,
            'quality': quality,
            'value': value,
            'momentum': momentum,
            'qvm': qvm
        },
        screen = min_liq_univ()
    )

In [79]:
# Specify a time range to evaluate
period_start = '2018-03-25'
period_end = '2018-03-25'

# Execute pipeline over evaluation period
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date=period_start,
    end_date=period_end
)

In [80]:
print('number of stocks: '+ str(len(pipeline_output)))
print('quality rank: ' + str(pipeline_output['quality'].min()) + ' to ' + str(pipeline_output['quality'].max()))
print('value rank: ' + str(pipeline_output['value'].min()) + ' to ' + str(pipeline_output['value'].max()))
print('momentum rank: ' + str(pipeline_output['momentum'].min()) + ' to ' + str(pipeline_output['momentum'].max()))
print('qvm rank: ' + str(pipeline_output['qvm'].min()) + ' to ' + str(pipeline_output['qvm'].max()))

number of stocks: 4370
quality rank: 1.0 to 3880.0
value rank: 1.0 to 4156.0
momentum rank: 1.0 to 4357.0
qvm rank: 1.0 to 3847.0


In [84]:
pipeline_output.sort_values(by = 'qvm', ascending = False).head(10)

fcf_yield  fcf_yield_manual  \
2018-03-26 00:00:00+00:00 Equity(33862 [GEN])      0.3859          0.384630   
                          Equity(47372 [SPKE])     0.4053          0.404443   
                          Equity(49678 [OSG])      0.1897          0.189693   
                          Equity(17730 [PTN])      0.1301          0.130089   
                          Equity(1068 [BPT])       0.1150          0.423176   
                          Equity(9094 [BBSI])      0.1756          0.175760   
                          Equity(27247 [XRM])      0.1095          0.109209   
                          Equity(40592 [WD])       0.6156          0.615219   
                          Equity(12765 [AOI])      0.0917          0.091650   
                          Equity(34238 [DHX])      0.2441          0.240712   

                                                      mktcap  momentum  \
2018-03-26 00:00:00+00:00 Equity(33862 [GEN])   2.471421e+08    4290.0   
                          Equity(47372 [SPKE])  3.981388e+08    4251.0   
                          Equity(49678 [OSG])   2.045240e+08    4309.0   
                          Equity(17730 [PTN])   2.188180e+08    4191.0   
                          Equity(1068 [BPT])    5.050400e+08    4157.0   
                          Equity(9094 [BBSI])   6.013453e+08    4195.0   
                          Equity(27247 [XRM])   1.072087e+08    4145.0   
                          Equity(40592 [WD])    1.638238e+09    3989.0   
                          Equity(12765 [AOI])   2.319747e+08    4240.0   
                          Equity(34238 [DHX])   9.073443e+07    4064.0   

                                                quality     qvm    ret_20  \
2018-03-26 00:00:00+00:00 Equity(33862 [GEN])    3850.0  3847.0  0.295833   
                          Equity(47372 [SPKE])   3838.0  3846.0  0.237205   
                          Equity(49678 [OSG])    3664.0  3845.0  0.366492   
                          Equity(17730 [PTN])    3793.0  3844.0  0.172775   
                          Equity(1068 [BPT])     3878.0  3843.0  0.148780   
                          Equity(9094 [BBSI])    3507.0  3842.0  0.177108   
                          Equity(27247 [XRM])    3791.0  3841.0  0.142609   
                          Equity(40592 [WD])     3527.0  3840.0  0.088103   
                          Equity(12765 [AOI])    3796.0  3839.0  0.223278   
                          Equity(34238 [DHX])    3378.0  3838.0  0.106061   

                                                      roe   value  
2018-03-26 00:00:00+00:00 Equity(33862 [GEN])    1.268190  4085.0  
                          Equity(47372 [SPKE])   0.786036  4090.0  
                          Equity(49678 [OSG])    0.187400  3957.0  
                          Equity(17730 [PTN])    0.388445  3806.5  
                          Equity(1068 [BPT])    18.701231  3741.0  
                          Equity(9094 [BBSI])    0.123732  3940.0  
                          Equity(27247 [XRM])    0.381490  3703.0  
                          Equity(40592 [WD])     0.130145  4122.0  
                          Equity(12765 [AOI])    0.394352  3537.0  
                          Equity(34238 [DHX])    0.093705  4022.0